In [111]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_25596\4143411418.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [128]:
from selenium.webdriver.common.keys import Keys
# import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import selenium
from cdp_backend.pipeline import ingestion_models
from datetime import datetime

In [156]:
sessions = []

# Get Datetime:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)

video_link = driver.find_element(by='id', value='VideoIcon_0_0')
video_link.click()

sessions.append(
    ingestion_models.Session(
        # Get Video Link:
        video_uri=driver.find_element(by='css selector', value='#existing-ytvideo-iframe').get_attribute('src'),
        session_index=0,
        session_datetime=dt
    )
)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#existing-ytvideo-iframe"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x01045FD3+2187219]
	Ordinal0 [0x00FDE6D1+1763025]
	Ordinal0 [0x00EF3E78+802424]
	Ordinal0 [0x00F21C10+990224]
	Ordinal0 [0x00F21EAB+990891]
	Ordinal0 [0x00F4EC92+1174674]
	Ordinal0 [0x00F3CBD4+1100756]
	Ordinal0 [0x00F4CFC2+1167298]
	Ordinal0 [0x00F3C9A6+1100198]
	Ordinal0 [0x00F16F80+946048]
	Ordinal0 [0x00F17E76+949878]
	GetHandleVerifier [0x012E90C2+2721218]
	GetHandleVerifier [0x012DAAF0+2662384]
	GetHandleVerifier [0x010D137A+526458]
	GetHandleVerifier [0x010D0416+522518]
	Ordinal0 [0x00FE4EAB+1789611]
	Ordinal0 [0x00FE97A8+1808296]
	Ordinal0 [0x00FE9895+1808533]
	Ordinal0 [0x00FF26C1+1844929]
	BaseThreadInitThunk [0x7627FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77947A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77947A6E+238]


In [153]:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)

datetime.datetime(2022, 6, 30, 9, 30)

In [122]:
# Inspecting each minute_item:
li = driver.find_elements(by='css selector', value='li.customLineHeight')
event_minutes_items = []
for list in li[:7]:
    try:
        if list.find_element(by='css selector', value='a').get_attribute('class') == 'hrefnowrap':
            link = list.find_element(by='css selector', value='.hrefnowrap')#.get_attribute('href')
            # link.click()
            print(link.text)
        else:
            try:
                model = ingestion_models.EventMinutesItem(
                    minutes_item = ingestion_models.MinutesItem(
                        list.find_element(by='css selector', value='.custom-rcaview-action-review').text
                    ),
                    decision = list.find_element(by='css selector', value='.col-md-12 b').text.replace('Action Taken: ', '')
                )
                event_minutes_items.append(model)
            except (selenium.common.exceptions.NoSuchElementException):
                continue
    except (selenium.common.exceptions.NoSuchElementException):
        continue
event_minutes_items
        

<class 'NoneType'>
<class 'NoneType'>


[EventMinutesItem(minutes_item=MinutesItem(name='Adoption of the agenda.', description=None, external_source_id=None), index=None, matter=None, supporting_files=None, decision='Adopted as amended', votes=None),
 EventMinutesItem(minutes_item=MinutesItem(name='Acceptance of minutes\nRegular Meeting of June 16, 2022', description=None, external_source_id=None), index=None, matter=None, supporting_files=None, decision='Accepted', votes=None),
 EventMinutesItem(minutes_item=MinutesItem(name='Referral of petitions, communications, and reports to the proper Committees.', description=None, external_source_id=None), index=None, matter=None, supporting_files=None, decision='Referred', votes=None)]

In [109]:
driver.quit()